In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import imageUtils
from utils import postProcessing
from utils import MultiDefectDetectionDataset
from utils import evaluation
from utils import visualization
import numpy as np
import os
from chainercv.links import FasterRCNNVGG16
from chainercv.visualizations import vis_bbox
from chainercv.utils import write_image
import chainer
import math
from chainercv import utils
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
#load Data
root = './data/3Types/Data3TypesYminXminYmaxXmax8'
dataset_train = MultiDefectDetectionDataset(data_dir=root, split='train')
dataset_NoAug = MultiDefectDetectionDataset(data_dir=root, split='NoImgAugtrain')
dataset_test = MultiDefectDetectionDataset(data_dir=root, split='test')
dataset_human = MultiDefectDetectionDataset(data_dir=root,split="human")
bbox_label_names = ('111', 'dot','100')

In [4]:
# DataSet Statistics
print('total number of training images: ', len(dataset_train))
print('total number of test images: ', len(dataset_test))
print('type of defects: ', bbox_label_names)

total number of training images:  1200
total number of test images:  15
type of defects:  ('111', 'dot', '100')


In [5]:
def getSATA(dataset,bbox_label_names):
    numKindsList = np.zeros(shape=(1, len(bbox_label_names)))
    for instance in dataset:
        img, gt_bboxes, gt_labels = instance
        gt_labels = gt_labels.tolist()
        for item in gt_labels:
            numKindsList[0,int(item)] += 1
    #numKindsList = numKindsList.tolist()
    print(numKindsList[0])
    print("Relative Probabilities")
    print(1.0 * numKindsList[0]/sum(1.0 * numKindsList[0]))
    return numKindsList

# test function
getSATA(dataset_human,bbox_label_names)

[ 172.  407.  161.]
Relative Probabilities
[ 0.23243243  0.55        0.21756757]


array([[ 172.,  407.,  161.]])

In [6]:
# Faster R-CNN model

use_gpu = False
# Higher NMS
proposal_params = {'min_size': 8,'nms_thresh':0.4}
model = FasterRCNNVGG16(n_fg_class=3,
                        pretrained_model='modelResults/snapshot_model_244000_Shufulle.npz',
                        ratios=[ 0.5, 1, 1.5, 2, 2.5, 4,8,16], anchor_scales=[1, 4, 8, 16], min_size=1024,
                        max_size=1024, proposal_creator_params=proposal_params)

model.score_thresh = 0.25

if use_gpu:
    chainer.cuda.get_device_from_id(0).use()
    model.to_gpu()

In [7]:
# name id convertion
id2name = {
1:"1ROI_100kx_4100CL_foil1.jpg",
2:"8ROI_100kx_4100CL_foil1.jpg",
3:"200kV_500kx_p2nm_8cmCL_grain1_0056 - Copy.jpg",
4:"200kV_500kx_p2nm_8cmCL_grain2_0036.jpg",
5:"0501_300kx_1nm_clhaadf3_0010.jpg",
6:"0501_300kx_1nm_clhaadf3_0014.jpg",
7:"0501_300kx_1nm_clhaadf3_0028.jpg",
8:"5401_300kx_1nm_clhaadf3_0020.jpg",
9:"BF X500K, 06 (2).jpg",
10:"BF X500K, 09 (3).jpg",
11:"dalong1.jpg",
12:"g1_backonzone_GBtowardsfrom_0007.jpg",
13:"g2_midonzone_GBtowardsfront_0010.jpg",
14:"grid1_roi1_500kx_0p5nm_haadf1_0025.jpg",
15:"grid1_roi2_500kx_0p5nm_haadf1_0047.jpg"
}

In [ ]:
def saveImage(img,fname):
    """
    :param img (numpy.ndarray): image in CHW format
    :return: plot the red channel in grayscale color map

    """
    plt.imshow(img.transpose((1, 2, 0))[:, :, 0], cmap='gray')
    plt.axis('off')
    plt.savefig(fname,dpi=150,bbox_inches='tight')
    

from utils.imageUtils import cropImage
for i in range(len(id2name)):
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(1, 1, 1)
    img, bbox, label = dataset_test.get_example_by_name(id2name[i+1].rstrip('.jpg'))
    visualization.vis_bbox(img, bbox, label, label_names=bbox_label_names, ax=ax1)
    os.mkdir(str(i+1))
    bboxes, labels, scores = model.predict([img])
    subimages, bboxes = cropImage(img, bboxes[0])
    for j in range(len(subimages)):
        imgName = str(i) + "/" + str(j)+'_'+ str(labels[0][j]) +".jpg"
        saveImage(subimages[j],imgName)